In [ ]:
from model import XLM_RoBERTa_classifier_one
from utils import get_datasets
from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score
from torch.optim import Adam
from torch import nn
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')

In [ ]:
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
#device = torch.device("cpu")
device

In [ ]:
train, test, val = get_datasets()

In [ ]:
train_dl, test_dl, val_dl = DataLoader(train, batch_size=128, shuffle=True), DataLoader(test, batch_size=128, shuffle=True), DataLoader(val, batch_size=128, shuffle=True)

In [ ]:
model = XLM_RoBERTa_classifier_one()
optimizer = Adam(model.parameters(), lr=5e-5)
model.to(device)
num_epochs = 3
loss_fun = nn.CrossEntropyLoss()

losses = []
acc = []


i = 0

for epoch in range(num_epochs):
    for batch in tqdm(train_dl):
        tok_batch = tokenizer(batch['premise'], batch['hypothesis'], return_tensors='pt', padding="max_length",
            max_length=64,
            truncation=True)
        labels = batch['label'].to(device)

        outputs = model(tok_batch['input_ids'].to(device), tok_batch['attention_mask'].to(device))
        loss = loss_fun(outputs, labels)
        acc_curr = accuracy_score(labels.cpu(), outputs.cpu().argmax(dim=1))
        acc.append(acc_curr)
        loss.backward()

        optimizer.step()
#         lr_scheduler.step()
        optimizer.zero_grad()
        # progress_bar.update(1)
        if i % 20 == 0:
            print("Train set accuracy: " + str(acc_curr))
        i = i + 1
        losses.append(loss.item())
    #torch.save(model.state_dict(), "/content/drive/MyDrive/xlm_roberta_state/xlm_roberta.pth")
    print(np.mean(losses))